In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#from _typeshed import Self
import torch
import torch.nn.functional
import torch.utils.data
import torchvision

import time
import copy
import torch
import torchvision

In [ ]:
data_transforms = {
    'train': torchvision.transforms.Compose([
        torchvision.transforms.Resize((240, 320)),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': torchvision.transforms.Compose([
        torchvision.transforms.Resize((240, 320)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
data_dir_path = '/content/drive/MyDrive/Colab_Notebooks/images/'
image_datasets = {
    'train': torchvision.datasets.ImageFolder(root=data_dir_path + 'TRAIN',
                                              transform=data_transforms['train']),
    'val':   torchvision.datasets.ImageFolder(root=data_dir_path + 'TEST',
                                              transform=data_transforms['val'])
}
dataloaders = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=4, shuffle=True),
    'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=4)
}
dataset_sizes = {
    'train': len(image_datasets['train']),
    'val': len(image_datasets['val'])
}
class_names = image_datasets['train'].classes

In [ ]:
class Net(torch.nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 16, 5)
        self.pool1 = torch.nn.MaxPool2d(2, 2)
        self.conv2 = torch.nn.Conv2d(16, 32, 5)
        self.pool2 = torch.nn.MaxPool2d(2, 2)
        self.fc1 = torch.nn.Linear(32 * 57 * 77, 512)
        self.fc2 = torch.nn.Linear(512, 64)
        self.fc3 = torch.nn.Linear(64, 4)


    def forward(self, x):
        x = torch.nn.functional.relu(self.conv1(x))
        x = self.pool1(x)
        x = torch.nn.functional.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 32 * 57 * 77)
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
net = Net()

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [ ]:
net.train()

Net(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=140448, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=4, bias=True)
)

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs):
    since = time.time()
    
    # save the best model during learning
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # training mode
            else:
                model.eval()   # evaluate mode

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)

                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)


            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

net = Net()
net = net.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model = train_model(net, dataloaders, criterion, optimizer, scheduler, num_epochs=50)

torch.save(self.model.state_dict(), weights_fpath)  

Epoch 0/49
----------
train Loss: 0.6460 Acc: 0.7050
val Loss: 0.1550 Acc: 1.0000
Epoch 1/49
----------
train Loss: 0.2176 Acc: 0.9117
val Loss: 0.1417 Acc: 1.0000
Epoch 2/49
----------
train Loss: 0.1172 Acc: 0.9517
val Loss: 0.0096 Acc: 1.0000
Epoch 3/49
----------
train Loss: 0.1166 Acc: 0.9583
val Loss: 0.7342 Acc: 0.5800
Epoch 4/49
----------
train Loss: 0.0309 Acc: 0.9867
val Loss: 0.3102 Acc: 0.8200
Epoch 5/49
----------
train Loss: 0.0401 Acc: 0.9850
val Loss: 0.0301 Acc: 1.0000
Epoch 6/49
----------
train Loss: 0.0451 Acc: 0.9850
val Loss: 0.5586 Acc: 0.7000
Epoch 7/49
----------
train Loss: 0.0083 Acc: 1.0000
val Loss: 0.4554 Acc: 0.8450
Epoch 8/49
----------
train Loss: 0.0029 Acc: 1.0000
val Loss: 0.4335 Acc: 0.8400
Epoch 9/49
----------
train Loss: 0.0023 Acc: 1.0000
val Loss: 0.3752 Acc: 0.8450
Epoch 10/49
----------
train Loss: 0.0015 Acc: 1.0000
val Loss: 0.3496 Acc: 0.8500
Epoch 11/49
----------
train Loss: 0.0012 Acc: 1.0000
val Loss: 0.3328 Acc: 0.8500
Epoch 12/49
--

NameError: ignored

# New section